In [13]:
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error


# Read the data
data = pd.read_csv(r'C:\Users\Denis\Desktop\py4e\Chapter16\Capstone\Kaggle\ML\Intermediate\train.csv', index_col='Id') 
data_test = pd.read_csv(r'C:\Users\Denis\Desktop\py4e\Chapter16\Capstone\Kaggle\ML\Intermediate\test.csv', index_col='Id')

# Select subset of predictors
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data.SalePrice
data.drop(['SalePrice'], axis=1, inplace=True)


# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in data.columns if
                    data[cname].nunique() < 10 and 
                    data[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in data.columns if 
                data[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X= data[my_cols].copy()
X_test = data_test[my_cols].copy()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

                   
model=XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X,y)
scores = -1 * cross_val_score(clf, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Average MAE score:", scores.mean())
preds=clf.predict(X_test)

output=pd.DataFrame({'Id': X_test.index, 'SalePrice': preds})
output.to_csv('Final_Submission.csv', index=False)

Average MAE score: 16442.093881100172
